In [23]:
import numpy as np

import adaptoctree.morton as morton
import adaptoctree.tree as tree
import adaptoctree.plotting as plotting

# Test logic

In [24]:
N = int(1e3)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 150

In [3]:
max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

In [4]:
unbalanced = tree.build(particles, max_level, max_num_particles)
unbalanced = np.unique(unbalanced)
depth = max(morton.find_level(unbalanced))

In [5]:
balanced = tree.balance(unbalanced, depth)

In [6]:
numba_balanced = tree.numba_balance(unbalanced, depth)

In [7]:
# Check numba balancing works
assert set(list(numba_balanced)) == balanced

balanced = numba_balanced

In [8]:
balanced_leaves = tree.assign_points_to_keys(particles, balanced, x0, r0)

In [9]:
# Test num particles constraint

_, counts = np.unique(balanced_leaves, return_counts=True)
assert np.all(counts) < max_num_particles

In [10]:
# Check for balancing condition

for i in balanced:
    for j in balanced:
        if (i != j) and morton.are_neighbours(i, j, x0, r0):
            diff = morton.find_level(i) - morton.find_level(j)
            assert diff <= 1

In [11]:
# Check for overlaps

for i, ki in enumerate(balanced):
    for j, kj in enumerate(balanced):
        if j != i:
            assert ki not in morton.find_ancestors(kj)            

# Benchmarking

In [34]:
N = int(1e7)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 100


max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

unbalanced = tree.build(particles, max_level, max_num_particles)
unbalanced = np.unique(unbalanced)

len(unbalanced)

300566

In [29]:
%timeit tree.build(particles, max_level, max_num_particles)

343 ms ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit tst = tree.balance(unbalanced, depth)

519 ms ± 9.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
_ = tree.numba_balance(unbalanced, depth)

In [22]:
%timeit tree.numba_balance(unbalanced, depth)

5.27 ms ± 47.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
a = tree.numba_balance(unbalanced, depth)

In [17]:
b = tree.balance(unbalanced, depth)

In [18]:
assert b == a

In [ ]:
len(a)